In [3]:
#!/usr/bin/env python3
import hashlib
from nacl.signing import SigningKey

def ed25519_sk_from_hex(seed_hex: str) -> SigningKey:
    seed_bytes = bytes.fromhex(seed_hex)
    assert len(seed_bytes) == 32
    return SigningKey(seed_bytes)

def pack_balance_nonce(balance: int, nonce: int) -> bytes:
    return hashlib.sha256(
        balance.to_bytes(8, "little") + nonce.to_bytes(8, "little")
    ).digest()

class MerkleTree:
    def __init__(self, leaves):
        n = 1
        while n < len(leaves):
            n <<= 1
        leaves = leaves + [b'\x00' * 32] * (n - len(leaves))
        self.levels = [leaves]
        self._build()

    def _build(self):
        level = self.levels[0]
        while len(level) > 1:
            nxt = []
            for i in range(0, len(level), 2):
                L = level[i]
                R = level[i+1]
                parent = hashlib.sha256(L + R).digest()
                nxt.append(parent)
            self.levels.insert(0, nxt)
            level = nxt

    @property
    def root(self) -> bytes:
        return self.levels[0][0]

    def get_proof(self, index: int) -> list[bytes]:
        proof = []
        idx = index
        for level in range(len(self.levels) - 1, 0, -1):
            sibling_idx = idx ^ 1
            proof.append(self.levels[level][sibling_idx])
            idx //= 2
        return proof

# Fixed deterministic keys
PRIVATE_KEY_A = "09e8e25175b8d6fd4b2e8e0fd3a2cf7809e8e25175b8d6fd4b2e8e0fd3a2cf78"
PRIVATE_KEY_B = "1eafbd2e3c134b7de405194c3e2e9b3b1eafbd2e3c134b7de405194c3e2e9b3b"

if __name__ == "__main__":
    sk_A = ed25519_sk_from_hex(PRIVATE_KEY_A)
    vk_A = sk_A.verify_key
    address_A = vk_A.encode().hex()

    sk_B = ed25519_sk_from_hex(PRIVATE_KEY_B)
    vk_B = sk_B.verify_key
    address_B = vk_B.encode().hex()

    accounts = {
        'A': {'address': address_A, 'balance': 100, 'nonce': 0},
        'B': {'address': address_B, 'balance': 50,  'nonce': 0},
        'C': {'address': 'cccccccccccccccccccccccccccccccc', 'balance': 75,  'nonce': 0},
        'D': {'address': 'dddddddddddddddddddddddddddddddd', 'balance': 25,  'nonce': 0},
    }

    # Build initial Merkle tree
    leaves = []
    for key in ['A', 'B', 'C', 'D']:
        acct = accounts[key]
        addr_bytes = bytes.fromhex(acct['address'].zfill(32))
        bal_nonce_hash = pack_balance_nonce(acct['balance'], acct['nonce'])
        leaf = hashlib.sha256(addr_bytes + bal_nonce_hash).digest()
        leaves.append(leaf)
    tree = MerkleTree(leaves)
    print(f"Initial state root: 0x{tree.root.hex()}")

    # Generate Merkle proof for A
    proof_A = tree.get_proof(0)
    print("Merkle proof for leaf index 0 (A):")
    for i, sib in enumerate(proof_A):
        print(f" Level {i}: 0x{sib.hex()}")

    # Create transaction without timestamp
    tx = {
        "sender":   accounts['A']['address'],
        "receiver": accounts['B']['address'],
        "amount":   10,
        "nonce":    accounts['A']['nonce'],
    }
    receiver_bytes = bytes.fromhex(tx['receiver'].zfill(32))
    msg_bytes = (
        receiver_bytes +
        tx['amount'].to_bytes(8, "little") +
        tx['nonce'].to_bytes(8, "little")
    )
    msg_hash = hashlib.sha256(msg_bytes).digest()

    # Sign the message
    sk = ed25519_sk_from_hex(PRIVATE_KEY_A)
    vk = sk.verify_key
    signature = sk.sign(msg_hash).signature
    tx['signature'] = signature.hex()
    tx['pubkey'] = vk.encode().hex()

    print("\nSigned transaction:")
    for k, v in tx.items():
        print(f"  {k}: {v}")

    # Update balances for new leaf computation
    amt = tx['amount']
    accounts['A']['balance'] -= amt
    accounts['B']['balance'] += amt
    accounts['A']['nonce'] += 1

    # Compute new Merkle root
    new_leaves = []
    for key in ['A', 'B', 'C', 'D']:
        acct = accounts[key]
        addr_bytes = bytes.fromhex(acct['address'].zfill(32))
        bal_nonce_hash = pack_balance_nonce(acct['balance'], acct['nonce'])
        leaf = hashlib.sha256(addr_bytes + bal_nonce_hash).digest()
        new_leaves.append(leaf)
    new_tree = MerkleTree(new_leaves)
    print(f"New state root:    0x{new_tree.root.hex()}")

    # Output test vector
    print("\nTest vector (hex):")
    print("old_root:", tree.root.hex())
    print("sender_addr:", accounts['A']['address'].zfill(32))
    print("balance:", 100)
    print("nonce:", 0)
    print("proof:", [sib.hex() for sib in proof_A])
    print("receiver_addr:", accounts['B']['address'].zfill(32))
    print("amount:", 10)
    print("tx_nonce:", 0)
    print("signature:", signature.hex())
    print("pubkey:", vk.encode().hex())
    print("privkey:", sk.encode().hex())


Initial state root: 0x81970bca2c37099dea8330693f68f3826e12dfd13d6063081d4311359b566f5a
Merkle proof for leaf index 0 (A):
 Level 0: 0xf820eeff6d37b8bd198d528e2cde8b668d390f43cbc148edc24484fc1f32cd46
 Level 1: 0xc5ef4aad837b3400fbdb36f46c4a4360dbc3c26060664f212e416e8a160e0876

Signed transaction:
  sender: 154099f64e0f45a5ab820cdc0e7fa2e2441125bce7a4b78373c7ddd3754ddb01
  receiver: bf935f2218690a38330a85735957b1f1105ff709136e0110dcb3837e3b42e85d
  amount: 10
  nonce: 0
  signature: 784b9eaad5ee209c59357e09e71d67ef052159e69db3942b74a8974aa4bf3c0147b846cb488993e6d98b6928101fe4d105bfa2e48294568dc2bef8c1301d700d
  pubkey: 154099f64e0f45a5ab820cdc0e7fa2e2441125bce7a4b78373c7ddd3754ddb01
New state root:    0xfacf7bf4220c46f2d50a7f39ee134978ddb63d0d51cdfe8d96a170295687a899

Test vector (hex):
old_root: 81970bca2c37099dea8330693f68f3826e12dfd13d6063081d4311359b566f5a
sender_addr: 154099f64e0f45a5ab820cdc0e7fa2e2441125bce7a4b78373c7ddd3754ddb01
balance: 100
nonce: 0
proof: ['f820eeff6d37b8bd198d